In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
import numpy as np
import pandas as pd
import pickle

from spike_and_slab_ser import SpikeSlabSER

### Simulated Example

Here we simulate summary statistics (zscores) on ~2000 SNPs across 10 tissues.

There are 4 causal SNPs in the sumation active it tissues[0-1], [2-5], [4-8], [0-8] respectively. The final tissue has no causal SNPs active in it.

In [ ]:
Sigma, causal_snps, tissue_membership, causal = pickle.load(open('../simulation_scripts/T10_simulation', 'rb'))
T, N = causal.shape

effectsize = 5  # zscore @ causal snps

Sigma_reg = (Sigma + np.eye(N)*1e-6) / (1+1e-6)
chol = np.linalg.cholesky(Sigma_reg)

# simulate data according to MVN distribution of marignal test statistics
Y = (effectsize * Sigma @ causal.T + chol @ np.random.normal(size=causal.T.shape)).T
X = Sigma_reg

### Setting up the model
There are three parameters you need to specify.
1. `K` is the max number of components.
2. `prior_activity` is an array of length `K` with the prior probability that a tissues is active in each component. 
3. `prior_variance` is the prior variance on the entries of the weight matrix. This parameters can have a large impact on the performance of the algorithm. The variation approximmation $q(\{w_{tk}, s_{tk}\})$ is a mixture of two gaussians-- an approximate posterior to $p(w_{tk} | s_{tk} = 1)$ and $p(w_{tk} | s_{tk} = 0) = p(w_{tk})$. It turns out that the variance of the first distribution is determined by `prior_variance`-- small setting will result in harder assignments

There are two pieces of data you need to provide to the model 
1. `X` is the $N \times N$ LD matrix
2. `Y` is a $T \times N$ matrix of z-scores for each SNP in T tissues

You can also provide labels $\text{snp_ids}$ and $\text{tissue_ids}$ that will make the output of the model easier to read

There is no reason the model can't support NAN entries in $Y$ but that needs to implemented


### Training the model

To train the model we use a forward selection scheme on the variational parameters. That is, we first train the model as if there were only one component with various initializations. We select the best model in terms of the evidence lower bound. We then train the model as through there were two components with various initializations, and again select the model with the best ELBO. We proceed until we are learning the full model with K models.

This approach is always increasing in the elbo. In the lth step we are simply performing coordinate ascent on the variaional parameters except that the last $(K-l)$ components are  fixed at a setting where they are irrelevant/inactive. While this is considerably slower than training everything at once, we hope it avoids a lot of poor local optima. Specifically, we hope this avoids the scenario where two components capture the same effect. At a forward step, the elbo will prefer a model that describes new areas of the data to an model that captures the same effect.

We also think this is necessary since the terminal state of the component is extremely sensitive to initialization of the weights. The simple solution is that for each initialization we assign a weight of 1 to one tissue and 0 to the others-- this initialization is destined to find a component that includes the active tissue. Do this across all tissues and select the best initialization.

`forward_fit(early_stop=True)` uses the forward selection type optimization scheme as described above. For now, I suggest using this as it seems to give the type of results we want. `early_stop` is an option that, if at a step of optimization you learn that a component is essentially inactive, we will zero out assigments for all future components and only fit the model one final time.

`fit()` just trains the model from one random initialization. It is much faster, but the results are much less stable since this model is quite sensitive to initialization of `weights` and `activity`

In [ ]:
K = 8
prior_activity = np.exp(-1*np.linspace(3, 3, K))
prior_variance = 5.0

model = SpikeSlabSER(
    X=X, Y=Y, K=K,
    snp_ids=np.arange(N), tissue_ids=np.arange(T),
    prior_activity=prior_activity,
    prior_variance=prior_variance
)

In [ ]:
%%time
model.forward_fit(early_stop=True)

### Plots

Example call of different plotting functions

`plot_component(thresh=0.1)` plots the probability of using a component, weights of each component per tissue, and the posterior categorical distributions of each active component, components which have $p(\text{tissue t active in component k}) > \text{thresh}$ for some tissue

`plot_predictions()` in the first row superimposes the observed z scores (black crosses) and expected value under the trained model (red circles), in the second row plots predictions against observed z scores

`plot_manhattan(component, thresh=0.0)` makes manhattan plots for a component, colored by r^2 with the top snp in that component. `thresh` is a minimum probability to include a tissue in the plot. Tissues are plotted in decreasing order of probability of using `component`

In [ ]:
model.plot_components()

In [ ]:
model.plot_predictions()

In [ ]:
model.plot_manhattan(component=0, thresh=0.5)

### Looking at inducing points

Each component squeezes the data through one SNP. With a linear kernel we can think of this as a linear regression over SNPs with a single feature-- the correlation with some other SNP $z$. We put a categorical prior on $z$. Looking at the posterior shows us likely causal SNPs for the signal captured by that component

`get_top_snp_per_component()` returns the top SNP per component and the probability assigned to that snp

`get_confidence_sets(alpha=0.9, thresh=0.1)` returns, for each component, the top weighted snps so that $\sum p_i > \alpha$, 'thresh' filters out components with maximum assignment probability $ < \text{thresh}$

In [ ]:
np.set_printoptions(precision=3, suppress=True)
top_snps, top_snp_probabilities = model.get_top_snp_per_component()
print('True causal snps {}\nTop SNP per component {}\nProbability of top SNP: {}'.format(causal_snps, top_snps, top_snp_probabilities))

In [ ]:
cs = model.get_confidence_sets(0.99, thresh=0.5)
print('\n'.join(['component {}: {}'.format(key, cs[key]) for key in cs.keys()]))

### Colocalization

`get_component_colocalization(component)` returns the probability under the model that two tissues both use `component`

`get_global_colocalization` returns the probability that pairs of tissues share at least one component in common

In [ ]:
model.get_component_colocalization(1)

In [ ]:
model.get_global_colocalzation()

### Real eQTL example

Now we show a quick run through a real eQTL exaple

In [ ]:
def get_inputs(zscore_path, ld_path, gene):
    """
    small helper function to read in data
    we also regulize our estimate of X by adding a small positive to diagonal and renormalizing
    this shouldnt do much to change the shape of the data distribution, just make it full rank
    """
    X = pd.read_csv(ld_path + gene, index_col=0)
    zscores = pd.read_csv(zscore_path + gene + '.zscore_matrix.txt', '\t', index_col=0)

    nan_snps = np.all(np.isnan(X.values), axis=1)
    X = X.iloc[~nan_snps].iloc[:, ~nan_snps]

    active_snps = np.isin(X.index, zscores.index)
    X = X.iloc[active_snps].iloc[:, active_snps]

    active_snps = np.isin(zscores.index, X.index)
    Y = zscores.iloc[active_snps]
    Y = Y.iloc[:, ~np.any(np.isnan(Y.values), 0)]
    

    tissues = Y.columns.values
    snp_ids = Y.index.values
    pos = np.array([int(snp_id.split('_')[1]) for snp_id in snp_ids])

    Y = Y.T.values
    X = X.values
    X = (X + np.eye(X.shape[0])*1e-6) / (1+1e-6)
    
    return X, Y, tissues, snp_ids

In [ ]:
gene = 'ENSG00000073464.11'
#gene = 'ENSG00000141644.17'
#gene = 'ENSG00000164904.17'

In [ ]:
ld_path = '../marios_correlation_matrices/'
zscoore_path = '../data/zscore_genes_for_Karl/'

X, Y, tissues, snp_ids = get_inputs(zscoore_path, ld_path, gene)
T, N = Y.shape

In [ ]:
%%time
K = 20
prior_activity = np.exp(-1*np.linspace(3, 10, K))
model = SpikeSlabSER(X, Y, K, snp_ids, tissues, prior_activity, 10.0)

model.weights = np.zeros_like(model.weights)
model.forward_fit(early_stop=True)

In [ ]:
model.plot_assignment_kl()

In [ ]:
model.active[:, 13].max()

In [ ]:
model.get_confidence_sets(thresh=0.35)

In [ ]:
model.plot_confidence_sets_ld(thresh=0.35)

In [ ]:
plt.plot(model.elbos)

In [ ]:
model.plot_predictions()

In [ ]:
model.plot_manhattan(component=0, thresh=0.3)

In [ ]:
component_colocs = {}
for k in range(K):
    probs = (model.active[:, k] * model.pi[:, k][:, None])
    coloc = np.zeros((T, T))
    for t1 in range(T):
        for t2 in range(T):
            coloc[t1, t2] = 1 - np.exp(np.sum(np.log(1 - probs[:, t1] * probs[:, t2])))
            coloc[t1, t2] = np.sum(1 - probs[:, t1] * probs[:, t2])
    component_colocs[k] = pd.DataFrame(coloc, index=model.tissue_ids, columns=model.tissue_ids)


In [ ]:
k = 4
np.stack([np.outer(model.active[:, k], model.active[:, k]) for k in range(K)])

In [ ]:
componentwise = model.get_component_colocalization()

In [ ]:
1 - np.exp(np.sum(np.log(1 - componentwise), axis=0))

In [ ]:
model.get_pip()

In [ ]:
t = 0
snp = 4

pip = np.zeros((N, T))
for t in range(T):
    for n in range(N):
        pip[n, t] = 1 - np.exp(np.sum(np.log(1 - model.pi[n] * model.active[t])))
        
return pip

In [ ]:
sns.heatmap(pip[pip.max(1) > 0.1])

In [ ]:
model.active[t]

In [ ]:
model.active

In [ ]:
model.weights.shape, np.isnan(model.Y).shape

In [ ]:
model.plot_manhattan(component=3, thresh=0.5)

In [ ]:
model.get_component_colocalization(0)

In [ ]:
model.get_global_colocalzation()